In [4]:
import pandas as pd
from functools import reduce

Extração dos dados

In [5]:
atletas = pd.read_excel('Athletes.xlsx')
treinadores = pd.read_excel('Coaches.xlsx')
etradas_generos = pd.read_excel('EntriesGender.xlsx')
medalhas = pd.read_excel('Medals.xlsx')
times = pd.read_excel('Teams.xlsx')

Transformação

In [6]:
# Atletas por modalidade e país

atletas_disc = atletas.groupby(['NOC', 'Discipline']) ['Name'].nunique()
atletas_disc = pd.DataFrame(atletas_disc).reset_index(drop=False).rename(columns={'Name': 'numero_atletas',
                                                                   'Discipline': 'modalidade',
                                                                   'NOC': 'pais'})



In [ ]:
# Treinadores por modalidade e país

treinadores_disc = treinadores.groupby(['NOC', 'Discipline']) ['Name'].nunique()
treinadores_disc = pd.DataFrame(treinadores_disc).reset_index(drop=False).rename(columns={'Name': 'numero_treinadores',
                                                                                          'Discipline': 'modalidade',
                                                                                          'NOC': 'pais'})
treinadores_disc

,pais,modalidade,numero_treinadores
0,Angola,Handball,1
1,Argentina,Artistic Swimming,1
2,Argentina,Basketball,1
3,Argentina,Football,1
4,Argentina,Handball,1
...,...,...,...
156,United States of America,Volleyball,5
157,United States of America,Water Polo,2
158,Venezuela,Basketball,6
159,Venezuela,Volleyball,4


In [8]:
# Times por gênero

times['Categoria'] = times.Event.apply(lambda x: 'Masculino' if 'Men' in x
                                        else ('Feminino' if 'Women' in x
                                            else ('Misto' if 'Mixed' in x
                                                else 'Outros')))
                                            

In [9]:
times.Categoria.value_counts()

Categoria
Masculino    299
Feminino     292
Misto         94
Outros        58
Name: count, dtype: int64

In [10]:

times_masc = pd.DataFrame(times[times.Categoria == 'Masculino'].groupby(['NOC', 'Discipline'])['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_masculinos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_fem = pd.DataFrame(times[times.Categoria == 'Feminino'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_femininos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_mistos = pd.DataFrame(times[times.Categoria == 'Misto'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'times_mistos',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})

times_outros = pd.DataFrame(times[times.Categoria == 'Outros'].groupby(['NOC', 'Discipline']) ['Name'].nunique()).\
reset_index().rename(columns={'Name': 'outros',
                              'Discipline': 'modalidade',
                              'NOC': 'pais'})


Juntando as tabelas

In [ ]:
#Utilizando a biblioteca reduce para unir todos os df's de uma só vez

dfs = [atletas_disc,
            treinadores_disc,
            times_masc,
            times_fem,
            times_mistos,
            times_outros]

df_final = reduce(lambda left, right: pd.merge(left,
                                               right, on=['pais', 'modalidade'],
                                               how='outer'),
                                               dfs).fillna(0)

# Converte colunas com números para int

cols = ['numero_atletas', 
        'numero_treinadores', 
        'times_masculinos', 
        'times_femininos', 
        'times_mistos', 
        'outros']

df_final[cols] = df_final[cols].astype(int)



,pais,modalidade,numero_atletas,numero_treinadores,times_masculinos,times_femininos,times_mistos,outros
0,Afghanistan,Athletics,2,0,0,0,0,0
1,Afghanistan,Shooting,1,0,0,0,0,0
2,Afghanistan,Swimming,1,0,0,0,0,0
3,Afghanistan,Taekwondo,1,0,0,0,0,0
4,Albania,Artistic Gymnastics,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2114,Zambia,Swimming,2,0,0,0,0,0
2115,Zimbabwe,Athletics,1,0,0,0,0,0
2116,Zimbabwe,Golf,1,0,0,0,0,0
2117,Zimbabwe,Rowing,1,0,0,0,0,0


Carregamento para o Big Query